# Скрипт для сбора данных участников группы вк и отбора по профессии  
### Программа состоит из двух частей: 
1. Сбор данных
2. Обработка полученных данных и создание Excel таблицы с итоговым результатом. 

Этот скрипт берет на вход id группы или ссылку на группу ВКонтакте.  Сначала происходит процесс сбора данных, который занимает определенное время, зависящее от объема группы и нагруженности серверов VK. (Например, [группа яндекса](https://vk.com/yandex) в 190+ тысяч человек  парсилась 7 минут. За это время можно сходить чаек попить). 

**Важный момент**: пока идет скачивание данных, переходить к следующему пункту нельзя - [GIL](https://ru.wikipedia.org/wiki/Global_Interpreter_Lock) не даст вам это сделать и программа выдаст ошибку. 

Короткий гайд по получению id группы:
1. Заходим в интересующую нас группу
2. Кликаем на любой пост группы
![alt text](https://pp.userapi.com/c848636/v848636679/78fc0/ZYv7aPdX8P8.jpg) 
3. Вот эти цифры после 'wall-' - это то, что мы ищем. Сам идентификатор поста (цифры после знака подчеркивания) нам не нужен.

После появления надписи в output надписи *Данные скачаны, теперь можно обрабатывать* можно переходить к следующему шагу.

Инструкция по получению токена *со съехавшими картинками* доступна по [ссылке](https://docs.google.com/document/d/1c7pM3gdiePBtLPsc7J4TeI6T1z6kL5XP_NBb02yYhVk/edit)
Оперативно получить свой токен - очень важная задача. Если 2 человека запустят одновременно 2 скрипта с разных компьютеров, но с одним токеном, то все крашнется. 

In [1]:
%%time
import requests

token = open('vk_token.txt' , 'r').read()

# owner_id = 49336104 
# https://vk.com/public49336104

#Следующий вариант для упрощения жизни людей. Не надо самому искать id группы, просто вставляешь ссылку и наслаждаешься

owner_id = requests.get('https://api.vk.com/method/utils.resolveScreenName', 
                 params = {'v':'5.75', 'screen_name':str(input('Вставьте URL группы ')).split('com/')[1],
                           'access_token':token}).json()['response']['object_id']

const = 200
# owner_id = input()

def get_members(owner_id):
    from collections import defaultdict
    s = defaultdict(list)
    r = requests.post('https://api.vk.com/method/execute.career?group_id='+
                      str(owner_id)+'&offset='+str(0)+'&v='+'5.73'+'&count='+str(const)
                      +'&access_token='+token).json()['response']
    ##################################################################################
    members_count = r[0] #количество участников
    data = r[1]
    print('В сообществе', owner_id, 'участников :',members_count)
    print('Обработка началась, не запускайте пока следующий шаг')
    for k,v in data.items():
        s[k].extend(v)
    if members_count>const:
        for offset in range(const, members_count, const):
            try:
                count = const
                r = requests.post('https://api.vk.com/method/execute.career?group_id='+
                                 str(owner_id)+'&offset='+str(offset)+'&v='+'5.73'+'&count='+
                                 str(count)+'&access_token='+token).json()['response'][1]
                for k,v in r.items():
                    s[k].extend(v)
            except Exception:
                pass
    print('Данные скачаны, теперь можно обрабатывать')
           
    return s


db = get_members(owner_id)


Вставьте URL группы https://vk.com/club161659827
В сообществе 161659827 участников : 8378
Обработка началась, не запускайте пока следующий шаг
Данные скачаны, теперь можно обрабатывать
Wall time: 15.5 s


####  Обработка данных
Следующий скрипт создает из массива всех пользователей группы датасет, в который попадают пользователи, прошедшие фильтрацию по столбцу [должность в разделе career](https://vk.com/dev/objects/user?f=1.%2B%D0%91%D0%B0%D0%B7%D0%BE%D0%B2%D1%8B%D0%B5%2B%D0%BF%D0%BE%D0%BB%D1%8F).

1. Критерии фильтрации задаются вами вручную, посредством перечисления названия должностей через **запятую**. 
Пример: *java, developer, hadoop*. 

2. Повторяющиеся id будут удалены, останутся только уникальные значения. 
3. Критериев для отбора может быть от одного до плюс бесконечности, но я рекомендую вам особо не мучить себя и компьютер. 

4. Дальше у вас высветится **количество найденных пользователей** и **меню ввода** для сохранения полученных значений в экселевскую таблицу. 
5. Если хотите, чтобы файл сохранился, введите на ваш вкус любой из символов: *y, да, 1*

6. После этого придумайте название для файла, который будет сохранен в ту же папку, где находится этот скрипт (по идее)
7. В конце, вне зависимости от ваших выборов будет показан полученный датафрейм. 

#### Итоговая таблица имеет следующие столбцы:
1. link - ссылка на пользователя 
2. name - имя и фамилия пользователя
3. pos - список указанных пользователем должностей 
4. com - список указанных пользователем компаний 
5. faculty - название вуза
6. age - возраст




In [2]:
import re
import pandas as pd
from datetime import date
year = date.today().year
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', 800)



def preparation():
    """Берет словарь из предыдущей части, создает датафрейм, 
    откуда удаляет строки, где не заполнено поле pos 
    и проводит с оставшимися строками различные манипуляции"""    
    df = pd.DataFrame.from_dict(db)
    a = df[pd.notnull(df['pos'])]
    a = a[list(db)].astype('str')
    a['age'] = year - pd.to_datetime(a['bdate'], errors = 'coerce').dt.year 
    a['link'] = 'https://vk.com/id' + a['ids'] 
    a['name'] = a['first_name'] +' '+ a['last_name']
    a.drop(['first_name', 'last_name', 'bdate'], axis = 1, inplace=True)
    return a


prepared = preparation()

def sorting(prepared):
    """Принимает на вход отфильтрованный датасет, 
    проводит поиск по должностям и предлагает записать результат в csv
    В любом случае возвращает итоговый датасет"""
    all_data = pd.DataFrame()
    position = str(input('Введите названия позиций через запятую: '))
    position = [x.strip() for x in position.split(',')]

    for pos in position:
        prepared[pos] = prepared['pos'].str.contains(pos, flags=re.IGNORECASE, regex=True)
        result = prepared[prepared[pos] == True]
        result.drop(pos, axis = 1, inplace = True)
        all_data = all_data.append(result, ignore_index=True)
    ########################################################
    all_data.drop_duplicates(subset = 'link', keep='first', inplace=True)
    all_data = all_data[['link', 'name',  'pos', 'com', 'faculty', 'age', 'city']]
    print('Подходящих по запросу пользователей: найдено {} штук'.format(len(all_data)))
    while True:
        try:
            choice = str(input('Хотите сохранить в эксель?[Y/N] ').lower())
        except ValueError:
            print("Попробуйте ещё раз, с первого раза не пошло")
            #better try again... Return to the start of the loop
            continue
        else:
            break


    if choice in ['y', 'да', '1']:
        writer = pd.ExcelWriter('{}.xlsx'.format(str(input('Введите название файла: '))))
        all_data.to_excel(writer,'Sheet1', index = False)
        writer.save()   
        print('Файл сохранен')    
    else:
        print('Работа программы завершена')
    return all_data

sorting(prepared)

Введите названия позиций через запятую: java
Подходящих по запросу пользователей: найдено 2 штук
Хотите сохранить в эксель?[Y/N] n
Работа программы завершена


,link,name,pos,com,faculty,age,city
0,https://vk.com/id167043974,Сергей Козельский,['Java Developer'],['Svitla Systems Inc.'],['ЧНУ им. Ю. Федьковича'],NaN,Черновцы
1,https://vk.com/id361831872,Бекжан Кудайбергенов,['Senior Java Developer'],[None],['КЭУ Казпотребсоюза'],NaN,Астана
